In [20]:
import re
import sys
import math
import pandas as pd
import mysql.connector
from datetime import date
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [21]:
data = [
    'hotel santika. terjangkau, strategis, dekat stasiun, mall, apartment, pasar',
    'hotel margonda. strategis, dekat stasiun, kampus, mewah nyaman',
    'hotel aviary bintaro. hotel bintang 4. mewah, bersih, nyaman, pelayanan terbaik tenang',
]
keywords = ["mewah, nyaman, dan terjangkau"]

In [22]:
df = pd.DataFrame(data, columns=["text"])
df = df.dropna()
dfCombine = df["text"]
df

,text
0,"hotel santika. terjangkau, strategis, dekat st..."
1,"hotel margonda. strategis, dekat stasiun, kamp..."
2,"hotel aviary bintaro. hotel bintang 4. mewah, ..."


In [23]:
vectorizer = TfidfVectorizer(norm=None)
countVectorizer = CountVectorizer()
tfidf_matrix = vectorizer.fit_transform(dfCombine)
tfidf_matrix_count = countVectorizer.fit_transform(dfCombine)
new_tfidf_vector = vectorizer.transform(keywords)
new_tfidf_vector_count = countVectorizer.transform(keywords)

In [38]:
idf_values = vectorizer.idf_
words = vectorizer.get_feature_names_out()
document_frequencies = tfidf_matrix.astype(bool).sum(axis=0).A1

In [39]:
columnsName = ["Word"]
totalDocuments = 0

indexColumn = 1
for index in range(len(data) + len(keywords)):
    if index == len(data):
        indexColumn = 1
    
    if index < len(data):
        columnsName.append("Dokumen " + str(indexColumn))
        totalDocuments = totalDocuments + 1
    else:
        columnsName.append("Keyword " + str(indexColumn))
    
    indexColumn = indexColumn + 1
    
print(tfidf_matrix_count)

  (0, 6)	1
  (0, 14)	1
  (0, 19)	1
  (0, 16)	1
  (0, 5)	1
  (0, 15)	1
  (0, 8)	1
  (0, 0)	1
  (0, 12)	1
  (1, 6)	1
  (1, 16)	1
  (1, 5)	1
  (1, 15)	1
  (1, 9)	1
  (1, 7)	1
  (1, 10)	1
  (1, 11)	1
  (2, 6)	2
  (2, 10)	1
  (2, 11)	1
  (2, 1)	1
  (2, 4)	1
  (2, 3)	1
  (2, 2)	1
  (2, 13)	1
  (2, 18)	1
  (2, 17)	1


In [41]:
printDataFrame = pd.DataFrame({
    columnsName[0]: words
})

indexColumnTemp = 0
for indexColumn, columnName in enumerate(columnsName):
    
    if indexColumn == 0:
        continue
    elif indexColumn > len(data):
        if indexColumn == len(data):
            indexColumnTemp = 0
            
        printDataFrame[columnName] = ""
        for word_index, word in enumerate(words):
            printDataFrame.at[word_index, columnName] = new_tfidf_vector_count[indexColumnTemp, word_index]
    else:
        printDataFrame[columnName] = ""
        for word_index, word in enumerate(words):
            printDataFrame.at[word_index, columnName] = tfidf_matrix_count[indexColumnTemp, word_index]
            
    indexColumnTemp += indexColumnTemp


# add new row
#printDataFrame.loc[len(printDataFrame)] = ["a"]

#for word_index, word in enumerate(words):

print(printDataFrame)
#for index, tf in enumerate(tfidf_matrix_count):
#    tfidf_matrix_count[index, word_index]

          Word Dokumen 1 Dokumen 2 Dokumen 3 Keyword 1
0    apartment         1         1         1         0
1       aviary         0         0         0         0
2       bersih         0         0         0         0
3      bintang         0         0         0         0
4      bintaro         0         0         0         0
5        dekat         1         1         1         0
6        hotel         1         1         1         0
7       kampus         0         0         0         0
8         mall         1         1         1         0
9     margonda         0         0         0         0
10       mewah         0         0         0         1
11      nyaman         0         0         0         1
12       pasar         1         1         1         0
13   pelayanan         0         0         0         0
14     santika         1         1         1         0
15     stasiun         1         1         1         0
16   strategis         1         1         1         0
17      te

In [27]:
for word_index, word in enumerate(words):
    print(str(word_index + 1) + ". Word: " + str(word))
    print("   (DF:" + str(document_frequencies[word_index]) + "  IDF:" + str(idf_values[word_index]) + ")")

1. Word: apartment
   (DF:1  IDF:1.6931471805599454)
2. Word: aviary
   (DF:1  IDF:1.6931471805599454)
3. Word: bersih
   (DF:1  IDF:1.6931471805599454)
4. Word: bintang
   (DF:1  IDF:1.6931471805599454)
5. Word: bintaro
   (DF:1  IDF:1.6931471805599454)
6. Word: dekat
   (DF:2  IDF:1.2876820724517808)
7. Word: hotel
   (DF:3  IDF:1.0)
8. Word: kampus
   (DF:1  IDF:1.6931471805599454)
9. Word: mall
   (DF:1  IDF:1.6931471805599454)
10. Word: margonda
   (DF:1  IDF:1.6931471805599454)
11. Word: mewah
   (DF:2  IDF:1.2876820724517808)
12. Word: nyaman
   (DF:2  IDF:1.2876820724517808)
13. Word: pasar
   (DF:1  IDF:1.6931471805599454)
14. Word: pelayanan
   (DF:1  IDF:1.6931471805599454)
15. Word: santika
   (DF:1  IDF:1.6931471805599454)
16. Word: stasiun
   (DF:2  IDF:1.2876820724517808)
17. Word: strategis
   (DF:2  IDF:1.2876820724517808)
18. Word: tenang
   (DF:1  IDF:1.6931471805599454)
19. Word: terbaik
   (DF:1  IDF:1.6931471805599454)
20. Word: terjangkau
   (DF:1  IDF:1.69314718

In [28]:
dataTf = []
for index, document in enumerate(data):
    dataTf.append([])
    for word_index, word in enumerate(words):
    	dataTf[index].append(tfidf_matrix_count[index, word_index])

        
currentLength = len(dataTf)
for index, document in enumerate(keywords):
	dataTf.append([])
	for word_index, word in enumerate(words):
		dataTf[currentLength].append(new_tfidf_vector_count[index, word_index])
	currentLength += currentLength


In [29]:
#term frequency

rows = []
for word_index, word in enumerate(words):
    columns = []
    columns.append(word)
    for index in range(len(dataTf)):
        columns.append(dataTf[index][word_index])

    rows.append(columns)
    
tfDataFrame = pd.DataFrame(rows, columns=columnsName)
    
print(tfDataFrame)


          Word  Dokumen 1  Dokumen 2  Dokumen 3  Keyword 1
0    apartment          1          0          0          0
1       aviary          0          0          1          0
2       bersih          0          0          1          0
3      bintang          0          0          1          0
4      bintaro          0          0          1          0
5        dekat          1          1          0          0
6        hotel          1          1          2          0
7       kampus          0          1          0          0
8         mall          1          0          0          0
9     margonda          0          1          0          0
10       mewah          0          1          1          1
11      nyaman          0          1          1          1
12       pasar          1          0          0          0
13   pelayanan          0          0          1          0
14     santika          1          0          0          0
15     stasiun          1          1          0         

In [30]:
rows = []
for word_index, word in enumerate(words):
    columns = []
    columns.append(word)
    columns.append(document_frequencies[word_index])
    columns.append(idf_values[word_index])
    rows.append(columns)
    
weightedDataFrame = pd.DataFrame(rows, columns=["Word", "DF", "IDF"])
print(weightedDataFrame)
#for word_index, word in enumerate(words):
#    print(word_index + 1, " Word:",  word, "  TF:", document_frequencies[word_index], "  IDF:", idf_values[word_index])

          Word  DF       IDF
0    apartment   1  1.693147
1       aviary   1  1.693147
2       bersih   1  1.693147
3      bintang   1  1.693147
4      bintaro   1  1.693147
5        dekat   2  1.287682
6        hotel   3  1.000000
7       kampus   1  1.693147
8         mall   1  1.693147
9     margonda   1  1.693147
10       mewah   2  1.287682
11      nyaman   2  1.287682
12       pasar   1  1.693147
13   pelayanan   1  1.693147
14     santika   1  1.693147
15     stasiun   2  1.287682
16   strategis   2  1.287682
17      tenang   1  1.693147
18     terbaik   1  1.693147
19  terjangkau   1  1.693147


In [31]:
#tf x idf
tfidfDataFrame = tfDataFrame


for index, tf in enumerate(tfDataFrame.iterrows()):
    for indexColumn, column in enumerate(columnsName):
        if indexColumn > 0:
            tfidfDataFrame.at[index, column] = float(tfidfDataFrame.loc[index, column]) * float(idf_values[index])
    
print(tfidfDataFrame)


          Word  Dokumen 1  Dokumen 2  Dokumen 3  Keyword 1
0    apartment   1.693147   0.000000   0.000000   0.000000
1       aviary   0.000000   0.000000   1.693147   0.000000
2       bersih   0.000000   0.000000   1.693147   0.000000
3      bintang   0.000000   0.000000   1.693147   0.000000
4      bintaro   0.000000   0.000000   1.693147   0.000000
5        dekat   1.287682   1.287682   0.000000   0.000000
6        hotel   1.000000   1.000000   2.000000   0.000000
7       kampus   0.000000   1.693147   0.000000   0.000000
8         mall   1.693147   0.000000   0.000000   0.000000
9     margonda   0.000000   1.693147   0.000000   0.000000
10       mewah   0.000000   1.287682   1.287682   1.287682
11      nyaman   0.000000   1.287682   1.287682   1.287682
12       pasar   1.693147   0.000000   0.000000   0.000000
13   pelayanan   0.000000   0.000000   1.693147   0.000000
14     santika   1.693147   0.000000   0.000000   0.000000
15     stasiun   1.287682   1.287682   0.000000   0.0000

In [32]:
for doc_index, doc in enumerate(dfCombine):
    #print("Document:", doc)
    tfidf_values = tfidf_matrix[doc_index].toarray()[0]
    for feature_index, tf_value in enumerate(tfidf_values):
        #if tf_value > 0:
        idf_value = idf_values[feature_index]
        df_value = document_frequencies[feature_index]
        #print("  Word:", feature_names[feature_index], "  DF:", df_value, "  TF:", tf_value, "  IDF:", idf_value)
    #print()

In [33]:
# norm l2
tfidf_matrix.toarray()[0, 5]

1.2876820724517808

In [34]:
new_tfidf_vector.toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.28768207, 1.28768207, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.69314718]])

In [35]:
rows = []
for word_index, word in enumerate(words):
    columns = []
    columns.append(word)
    for index, matrix in enumerate(tfidf_matrix.toarray()):
        columns.append(matrix[word_index])
    
    for index, matrix in enumerate(new_tfidf_vector.toarray()):
        columns.append(matrix[word_index])

    rows.append(columns)
    
tfidfNormL2DataFrame = pd.DataFrame(rows, columns=columnsName)
print(tfidfNormL2DataFrame)

          Word  Dokumen 1  Dokumen 2  Dokumen 3  Keyword 1
0    apartment   1.693147   0.000000   0.000000   0.000000
1       aviary   0.000000   0.000000   1.693147   0.000000
2       bersih   0.000000   0.000000   1.693147   0.000000
3      bintang   0.000000   0.000000   1.693147   0.000000
4      bintaro   0.000000   0.000000   1.693147   0.000000
5        dekat   1.287682   1.287682   0.000000   0.000000
6        hotel   1.000000   1.000000   2.000000   0.000000
7       kampus   0.000000   1.693147   0.000000   0.000000
8         mall   1.693147   0.000000   0.000000   0.000000
9     margonda   0.000000   1.693147   0.000000   0.000000
10       mewah   0.000000   1.287682   1.287682   1.287682
11      nyaman   0.000000   1.287682   1.287682   1.287682
12       pasar   1.693147   0.000000   0.000000   0.000000
13   pelayanan   0.000000   0.000000   1.693147   0.000000
14     santika   1.693147   0.000000   0.000000   0.000000
15     stasiun   1.287682   1.287682   0.000000   0.0000

In [36]:
similarity_scores = cosine_similarity(new_tfidf_vector, tfidf_matrix)
best_match_index = similarity_scores.argmax()

vacanciesWeighted = similarity_scores[0]
sortedIndexVacancies = vacanciesWeighted.argsort()[::-1]

best_match_index

1

In [37]:
for index, document in enumerate(data):
	print("Document", index, "score: ", vacanciesWeighted[index], "rank: ", sortedIndexVacancies[index] + 1)

Document 0 score:  0.25583214172871 rank:  2
Document 1 score:  0.3440750866990368 rank:  1
Document 2 score:  0.25486102212125017 rank:  3
